# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [39]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [40]:
from chi import hardware

node_type = "gpu_p100"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 1 gpu_p100 nodes ready to use


### Reserve node

In [41]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [42]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,52c2de97-985b-4534-87ad-6d6f90753f57
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.128 (v4) Type: fixed MAC: 44:a8:42:27:0c:1e
Network Name,sharednet1
Created At,2025-03-30T15:10:52Z
Keypair,trovi-ef7b87a
Reservation Id,e7e8f40d-60c4-4d27-85e6-0e91ec6932f5
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [43]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.67 port 22.


Connection successful


### Setup nvidia drivers and cuda

Download drivers

In [48]:
my_server.execute("sudo apt update && sudo apt install -y nvidia-driver-525")
my_server.execute("wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb")
my_server.execute("sudo dpkg -i cuda-keyring_1.0-1_all.deb")
my_server.execute("sudo apt update")
my_server.execute("sudo apt install -y cuda-toolkit-12-2") 

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2423 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2181 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/main amd64 c-n-f Metadata [18.5 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3179 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 c-n-f Metadata [676 B]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1196 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates/universe amd64 c-n-f Metada

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libegl-mesa0
  libegl1 libfontenc1 libgbm1 libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0
  libglx-mesa0 libglx0 libice6 libllvm15 libnvidia-cfg1-535
  libnvidia-common-535 libnvidia-compute-535 libnvidia-decode-535
  libnvidia-encode-535 libnvidia-extra-535 libnvidia-fbc1-535 libnvidia-gl-535
  libpciaccess0 libsensors-config libsensors5 libsm6 libwayland-client0
  libx11-xcb1 libxaw7 libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0
  libxcb-shm0 libxcb-sync1 libxcb-xfixes0 libxcvt0 libxfixes3 libxfont2
  libxkbfile1 libxmu6 libxpm4 libxshmfence1 libxt6 libxxf86vm1
  nvidia-compute-utils-535 nvidia-dkms-535 nvidia-driver-535
  nvidia-firmware-535-535.183.01 nvidia-kernel-common-535
  nvidia-kernel-source-535 nvidia-utils-535 x11-common x11-xkb-utils
  xserver-common xserver-xorg-core 

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 363 MB in 9s (40.5 MB/s)
Selecting previously unselected package libdrm-amdgpu1:amd64.
(Reading database ... 84249 files and directories currently installed.)
Preparing to unpack .../00-libdrm-amdgpu1_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-amdgpu1:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libpciaccess0:amd64.
Preparing to unpack .../01-libpciaccess0_0.16-3_amd64.deb ...
Unpacking libpciaccess0:amd64 (0.16-3) ...
Selecting previously unselected package libdrm-intel1:amd64.
Preparing to unpack .../02-libdrm-intel1_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-intel1:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libdrm-nouveau2:amd64.
Preparing to unpack .../03-libdrm-nouveau2_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-nouveau2:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libdrm-radeon1:amd64.
Preparing to unpack .../04-libdrm-radeon1_2.4.

--2025-03-30 15:32:50--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.221.22.110, 23.221.22.103
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.221.22.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4328 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.0-1_all.deb’

     0K ....                                                  100%  148M=0s

2025-03-30 15:32:52 (148 MB/s) - ‘cuda-keyring_1.0-1_all.deb’ saved [4328/4328]



...

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [2068 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 2595 kB in 1s (2311 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
128 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  ca-certificates-java cuda-cccl-12-2 cuda-command-line-tools-12-2
  cuda-compiler-12-2 cuda-crt-12-2 cuda-cudart-12-2 cuda-cudart-dev-12-2
  cuda-cuobjdump-12-2 cuda-cupti-12-2 cuda-cupti-dev-12-2 cuda-cuxxfilt-12-2
  cuda-documentation-12-2 cuda-driver-dev-12-2 cuda-gdb-12-2
  cuda-libraries-12-2 cuda-libraries-dev-12-2 cuda-nsight-12-2
  cuda-nsight-compute-12-2 cuda-nsight-systems-12-2 cuda-nvcc-12-2
  cuda-nvdisasm-12-2 cuda-nvml-dev-12-2 cuda-nvprof-12-2 cuda-nvprune-12-2
  cuda-nvrtc-12-2 cuda-nvrtc-dev-12-2 cuda-nvtx-12-2 cuda-nvvm-12-2
  cuda-nvvp-12-2 cuda-opencl-12-2 cuda-opencl-dev-12-2 cuda-profiler-api-12-2
  cuda-sanitizer-12-2 cuda-toolkit-12-2-config-common
  cuda-toolkit-12-config-common cuda-toolkit-config-common cuda-tools-12-2
  cuda-visual-tools-12-2 default-jre default-jre-headless fontconfig-config
  fonts-dejavu-core gds-tools-1

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 2865 MB in 3min 5s (15.5 MB/s)
Selecting previously unselected package java-common.
(Reading database ... 85310 files and directories currently installed.)
Preparing to unpack .../00-java-common_0.72build2_all.deb ...
Unpacking java-common (0.72build2) ...
Selecting previously unselected package libavahi-common-data:amd64.
Preparing to unpack .../01-libavahi-common-data_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common-data:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-common3:amd64.
Preparing to unpack .../02-libavahi-common3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-common3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libavahi-client3:amd64.
Preparing to unpack .../03-libavahi-client3_0.8-5ubuntu5.2_amd64.deb ...
Unpacking libavahi-client3:amd64 (0.8-5ubuntu5.2) ...
Selecting previously unselected package libcups2:amd64.
Preparing to unpack .../04-libcups2_2.4.1op1-1ubuntu4.11_amd64.deb ...
Unpacking libcups2:amd6

<Result cmd='sudo apt install -y cuda-toolkit-12-2' exited=0>

Restart nvidia services

In [60]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

update-initramfs: Generating /boot/initrd.img-5.15.0-124-generic


<Result cmd='sudo modprobe nvidia' exited=0>

Verifiy services

In [67]:
my_server.execute("nvidia-smi")
my_server.execute("sudo ln -s /usr/local/cuda-12.2/bin/nvcc /usr/local/bin/nvcc")
my_server.execute("nvcc --version")

Sun Mar 30 15:55:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:03:00.0 Off |                    0 |
| N/A   25C    P0              30W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

<Result cmd='nvcc --version' exited=0>

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [68]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

In [ ]:
my_server.execute("chmod +x Fairify/reproduce.sh")
my_server.execute("bash Fairify/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

2025-03-30 15:57:18.928434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-30 15:57:18.928466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Number of partitions:  201


Processing Models:   0%|          | 0/5 [00:00<?, ?it/s]2025-03-30 15:57:20.135497: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2025-03-30 15:57:21.270673: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-03-30 15:57:21.270714: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: radhofanazizi-gmail-com-power-management
2025-03-30 15:57:21.270723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: radhofanazizi-gmail-com-power-management
2025-03-30 15:57:21.270876: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.183.1
2025-03-30 15:57:21.270910: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.183.1
2025-03-30 15:57:21.270919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel vers

==================  STARTING MODEL GC-5.h5
###################



Processing Partitions:   1%|          | 2/201 [03:25<4:41:34, 84.89s/it]  